In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

class style :
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# check python bit version
import platform
print(platform.architecture())

!pip install jupyternotify
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

!pip install email-notify-magic
%load_ext email_notify_magic

('64bit', '')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
     |████████████████████████████████| 120 kB 39.6 MB/s 
     |████████████████████████████████| 82 kB 715 kB/s 
  Created wheel for jupyternotify: filename=jupyternotify-0.1.15-py3-none-any.whl size=8741 sha256=573a0a21776943b22f444c3041568076288449a35968c92d1113cfd7f77dbecd
  Stored in directory: /root/.cache/pip/wheels/18/31/59/466711d8f86811d1b57f55f114d0516419f0093435f41c65d9
Successfully built jupyternotify


<IPython.core.display.Javascript object>

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import the package
import stanza
nlp = stanza.Pipeline('en')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.0 MB/s 
     |████████████████████████████████| 216 kB 47.1 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=26e6c691a9cb6fede84426b13688a478e3633a51551d6fe6eb67cfb0aa312758
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
%%notify
%%time
Review = pd.read_csv(r'resturant.csv',encoding = "utf-8")
Review.head(5)

CPU times: user 9.96 ms, sys: 2.68 ms, total: 12.6 ms
Wall time: 15.4 ms


,text,length,ind,aspects,polarities
0,Service is fast and friendly.,1,0,service,positive
1,I HATE HATE HATE this place.,1,0,anecdotes/miscellaneous,negative
2,Had we been stalling I could understand where ...,1,0,service,negative
3,"after dining at B.O.G over twenty times now, i...",1,0,anecdotes/miscellaneous,neutral
4,"The waitstaff were attentive, polite and helpf...",1,0,service,positive


<IPython.core.display.Javascript object>

In [6]:

Review= Review[['text', 'aspects', 'polarities']]
print(Review.shape)

(2604, 3)


## Cleaing: Code

In [7]:
%%notify
%%time

from nltk.corpus import wordnet
from nltk import word_tokenize

def clean(text):
    
    text_token = word_tokenize(text)
    text_token2 = [x.lower() for x in text_token]
    
    # Mapping Social-media shortcuts
    shortcuts = {'u': 'you','lol': 'laughing out loud', 'k': 'okay', 'm': 'am', 'b4': 'before',
                'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love','gr8': 'great', 'bday': 'birthday',
                 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate','irl': 'in real life', 'yolo': 'you only live once','btw': 'by the way'}
    nsw=pd.Series(text_token)
    words=nsw.map(shortcuts).fillna(nsw)
    
    #Get an integrated list of listed_words (words like yolo must be combined, You Only Live Once :)
    words= [word.split(' ') for word in words]    
    import itertools
    words= list(itertools.chain.from_iterable(words))
    
    # lemmatize the words with appropriate POS Tag
    def pos_tag(word):  
        tag= nltk.pos_tag([word])[0][1][0].upper()
        dic= {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
        return dic.get(tag, wordnet.NOUN)
    words = [WordNetLemmatizer().lemmatize(word, pos=pos_tag(word)) for word in words] 
    words2=[word for word in words if nltk.pos_tag([word])[0][1] not in ['VB','VBG','VBD','VBN','VBP','VBZ','CC','MD','IN']]
    
    '''# perform stemming for non lemmatized words (like 'cleaning')
    from nltk.stem.snowball import SnowballStemmer
    
    englishStemmer=SnowballStemmer("english")
    words=[englishStemmer.stem(word) for  word in words]'''
    
    '''words=[word.encode('utf-8') for word in words]
    words=[hnspl.suggest(word)[0] if hnspl.spell(word)==False else word for word in words]'''
    
    clean_text=" ".join(words2)
    return clean_text

CPU times: user 301 ms, sys: 102 ms, total: 403 ms
Wall time: 967 ms


<IPython.core.display.Javascript object>

## Stanford&Stanza: ABSA

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize 

nltk.download('stopwords')
nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
stanza.download("en")
nlp = stanza.Pipeline('en') # initialize English neural pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [15]:
import stanza
nlp = stanza.Pipeline('en') # initialize pipeline
def ABSA(txt):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word
        #taggedList = pos_tagger.tag(txt_list)
        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        #wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(new_txt_list)

        doc = nlp(finaltxt) # Object of Stanza NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return(finalcluster)
import nltk


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## Test 

In [16]:
txt = "Service is fast and friendly."
ABSA(txt)

[['service', ['fast']]]

In [17]:
txt1 = "Had we been stalling I could understand where they were coming from, but we had been there less than an hour and they hadn't even brought us a check yet!"

ABSA(txt1)

[['i', ['understand']], ['hour', ['less']], ['check', ['brought']]]